In [1]:
# === FLAT (filtered) -> NLQ ufficiale (compatibile prepare) ===
import json, os, math, hashlib
from collections import defaultdict

# --- CONFIG ---
IN_PATH  = "nlq_pretrain.json"
OUT_PATH = "nlq_pretrain.json"



def sfloat(x, default=None):
    try: return float(x)
    except: return default

def mk_clip_uid(video_uid, cs, ce, idx):
    base = f"{video_uid}_{int(math.floor(cs or 0))}_{int(math.floor(ce or 0))}_{idx}"
    return base.replace(" ", "_")

def mk_ann_uid(clip_uid, ann_idx=0):
    return f"{clip_uid}-ann{ann_idx}"

def mk_query_id(video_uid, clip_uid, local_idx):
    raw = f"{video_uid}|{clip_uid}|{local_idx}"
    return hashlib.md5(raw.encode("utf-8")).hexdigest()[:16]

def get_query_times(item, cs, ce):
    """
    Ordine di priorità: video_*  -> query_* -> start/end.
    Clamp dentro [cs, ce] e corregge eventuali swap.
    """
    cands = [
        (item.get("video_start_sec"), item.get("video_end_sec")),
        (item.get("query_start_sec"), item.get("query_end_sec")),
        (item.get("start_sec"),       item.get("end_sec")),
    ]
    qs = qe = None
    for a, b in cands:
        qa, qb = sfloat(a), sfloat(b)
        if qa is not None and qb is not None:
            qs, qe = qa, qb
            break
    if qs is None or qe is None:
        qs, qe = cs, ce
    if qe < qs:
        qs, qe = qe, qs
    # clamp nella clip
    qs = max(cs, min(qs, ce))
    qe = max(cs, min(qe, ce))
    # evita finestra degenerata
    if qe == qs:
        mid = (cs + ce) / 2.0
        eps = min(0.01, max(1e-3, (ce - cs) * 1e-4))
        qs = max(cs, mid - eps)
        qe = min(ce, mid + eps)
    return float(qs), float(qe)

# --- Carica flat filtrato ---
with open(IN_PATH, "r", encoding="utf-8") as f:
    flat = json.load(f)

# --- Raggruppa per (video_uid, clip_start_sec, clip_end_sec) ---
groups = defaultdict(list)
for it in flat:
    vid = it.get("video_uid")
    cs  = sfloat(it.get("clip_start_sec"))
    ce  = sfloat(it.get("clip_end_sec"))
    if not vid or cs is None or ce is None:
        continue
    if ce < cs:
        cs, ce = ce, cs
    groups[(str(vid), cs, ce)].append(it)

# --- Costruisci struttura NLQ ---
videos = defaultdict(list)
per_video_counter = defaultdict(int)

kept_queries = 0
for (vid, cs, ce), items in groups.items():
    idx = per_video_counter[vid]; per_video_counter[vid] += 1
    clip_uid = mk_clip_uid(vid, cs, ce, idx)

    lqs = []
    for i, x in enumerate(items):
        qtext = (x.get("query") or "").strip()
        if not qtext:
            continue
        qs, qe = get_query_times(x, cs, ce)
        lqs.append({
            "query": qtext,
            "template": x.get("template"),
            "template_id": int(x["template_id"]) if x.get("template_id") is not None else None,
            # richiesti in molti prepare:
            "clip_start_sec": float(cs),
            "clip_end_sec":   float(ce),
            "video_start_sec": float(qs),
            "video_end_sec":   float(qe),
            "rationale": x.get("rationale"),
            "query_id": mk_query_id(vid, clip_uid, i),
        })
    if not lqs:
        continue

    videos[vid].append({
        "clip_uid": clip_uid,
        "clip_start_sec": float(cs),
        "clip_end_sec":   float(ce),
        "video_start_sec": float(cs),
        "video_end_sec":   float(ce),
        "annotations": [{
            "annotation_uid": mk_ann_uid(clip_uid, 0),
            "language_queries": lqs
        }]
    })
    kept_queries += len(lqs)

nlq = {
    "videos": [{"video_uid": vid, "clips": clips} for vid, clips in videos.items()]
}

with open(OUT_PATH, "w", encoding="utf-8") as f:
    json.dump(nlq, f, ensure_ascii=False, indent=2)

print(f"[OK] NLQ scritto: {OUT_PATH}")
print("  - #videos :", len(nlq["videos"]))
print("  - #clips  :", sum(len(v['clips']) for v in nlq['videos']))
print("  - #queries:", sum(len(a['language_queries']) for v in nlq['videos'] for c in v['clips'] for a in c['annotations']))

# sanity: mostra le chiavi del primo datum
for v in nlq["videos"][:1]:
    for c in v["clips"][:1]:
        ex = c["annotations"][0]["language_queries"][0]
        print("Esempio language_query keys:", sorted(ex.keys()))
        break
    break

AttributeError: 'str' object has no attribute 'get'

In [1]:
%%bash

BASE="/media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning"
REPO="$BASE/episodic-memory/NLQ/VSLNet"
DATA="$BASE/ego4d_data"

mkdir -p "$REPO/data/features/nlq_official_v1_omnivore/official"
mkdir -p "$REPO/data/dataset/nlq_official_v1_omnivore"

cd "$REPO/utils"
python3 prepare_ego4d_dataset.py \
  --input_train_split "$BASE/nlq_pretrain.json" \
  --input_val_split   "$DATA/v1/annotations/nlq_val.json" \
  --input_test_split  "$DATA/v1/annotations/nlq_test_unannotated.json" \
  --video_feature_read_path "$DATA/v1/omnivore_video_swinl_fp16/v1/omnivore_video_swinl_fp16/" \
  --clip_feature_save_path  "$REPO/data/features/nlq_pretrain_v1_omnivore/official" \
  --output_save_path        "$REPO/data/dataset/nlq_pretrain_v1_omnivore"


Reading [train]: /media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/nlq_pretrain.json
# train: 49714
Writing [train]: /media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/episodic-memory/NLQ/VSLNet/data/dataset/nlq_pretrain_v1_omnivore/train.json
Reading [val]: /media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/ego4d_data/v1/annotations/nlq_val.json
# val: 3874
Writing [val]: /media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/episodic-memory/NLQ/VSLNet/data/dataset/nlq_pretrain_v1_omnivore/val.json
Reading [test]: /media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/ego4d_data/v1/annotations/nlq_test_unannotated.json
# test: 4004
Writing [test]: /media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/episodic-memory/NLQ/VSLNet/data/dataset/nlq_pretrain_v1_omnivore/test.json
Inizio estrazione features con RALLENTAMENTO PROTETTIVO...


Extracting features: 100%|██████████| 13401/13401 [16:01<00:00, 13.94it/s]


In [2]:
%%bash
BASE="/media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning"
REPO="$BASE/episodic-memory/NLQ/VSLNet"
mkdir -p "$REPO/data/features/nlq" "$REPO/data/dataset"


# link a feature e dataset del tuo tag
ln -sfn "$REPO/data/features/nlq_pretrain_v1_omnivore/official" "$REPO/data/features/nlq/official"
ln -sfn "$REPO/data/dataset/nlq_pretrain_v1_omnivore" "$REPO/data/dataset/nlq"

# verifica esistenza file chiave
ls -l "$REPO/data/features/nlq/official/feature_shapes.json"
ls -l "$REPO/data/dataset/nlq/train.json" "$REPO/data/dataset/nlq/val.json"



-rw-rw-r-- 1 nicolo nicolo 747858 dic 13 17:02 /media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/episodic-memory/NLQ/VSLNet/data/features/nlq/official/feature_shapes.json
-rw-rw-r-- 1 nicolo nicolo 10357136 dic 13 16:46 /media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/episodic-memory/NLQ/VSLNet/data/dataset/nlq/train.json
-rw-rw-r-- 1 nicolo nicolo   496981 dic 13 16:46 /media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/episodic-memory/NLQ/VSLNet/data/dataset/nlq/val.json


In [ ]:
%%bash
BASE="/media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning"
REPO="$BASE/episodic-memory/NLQ/VSLNet"
MODEL_SUBDIR="$BASE/checkpoints/omnivore_vslnet_pretrain"

export DATALOADER_WORKERS=1
export NUM_WORKERS=2
export BATCH_SIZE=32
export DIM=128
export NUM_EPOCH=10
export MAX_POS_LEN=128
export INIT_LR=0.0001
export VAL_JSON_PATH="/home/nicolo/ingegneriaMatematica/machineLearning/ego4d_data/v1/annotations/nlq_val.json"
export TB_LOG_NAME="pretrain_bs${BATCH_SIZE}_dim${DIM}_ep${NUM_EPOCH}_lr${INIT_LR}"

cd "$REPO"
python main.py \
  --model_name vslnet \
  --task nlq \
  --predictor bert \
  --dim $DIM \
  --mode train \
  --video_feature_dim 1536 \
  --max_pos_len $MAX_POS_LEN \
  --init_lr $INIT_LR \
  --epochs $NUM_EPOCH \
  --batch_size $BATCH_SIZE \
  --fv official \
  --num_workers $NUM_WORKERS \
  --data_loader_workers $DATALOADER_WORKERS \
  --model_dir "$MODEL_SUBDIR" \
  --eval_gt_json "$VAL_JSON_PATH" \
  --log_to_tensorboard $TB_LOG_NAME \
  --tb_log_freq 5 \
  --remove_empty_queries_from train \
  --freeze_encoder_epochs 1


Running with Namespace(save_dir='datasets', task='nlq', eval_gt_json='/home/nicolo/ingegneriaMatematica/machineLearning/ego4d_data/v1/annotations/nlq_val.json', fv='official', max_pos_len=128, num_workers=2, data_loader_workers=1, word_size=None, char_size=None, word_dim=300, video_feature_dim=1536, char_dim=50, dim=128, highlight_lambda=5.0, num_heads=8, drop_rate=0.2, predictor='bert', gpu_idx='0', seed=12345, mode='train', epochs=10, batch_size=32, num_train_steps=None, init_lr=0.001, clip_norm=1.0, warmup_proportion=0.0, extend=0.1, period=100, text_agnostic=False, video_agnostic=False, model_dir='/media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/checkpoints/omnivore_vslnet_pretrain', model_name='vslnet', suffix=None, log_to_tensorboard='pretrain_bs32_dim128_ep10_lr0.001', tb_log_dir='./runs', tb_log_freq=5, slurm=False, slurm_wait=False, slurm_partition='pixar', slurm_constraint='volta', slurm_gpus=1, slurm_cpus=10, slurm_timeout_min=720, slurm_log_folder='slurm_l

process episodic nlq train: 100%|██████████| 12740/12740 [00:00<00:00, 305876.68it/s]


train: skipped = 0, remaining = 49714


process episodic nlq val: 100%|██████████| 328/328 [00:00<00:00, 111368.23it/s]


val: skipped = 0, remaining = 3874


...


evaluate val: 100%|██████████| 122/122 [00:07<00:00, 16.80it/s]


Evaluated: 3874 / 3875 instances
+Epoch 7, Step 10101--+-----------+----------+----------+-----------+----------+----------+-----------+------+
|  Rank@1  |  Rank@1  |   Rank@1  |  Rank@3  |  Rank@3  |   Rank@3  |  Rank@5  |  Rank@5  |   Rank@5  | mIoU |
| mIoU@0.3 | mIoU@0.5 | mIoU@0.01 | mIoU@0.3 | mIoU@0.5 | mIoU@0.01 | mIoU@0.3 | mIoU@0.5 | mIoU@0.01 |      |
+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+------+
|   1.24   |   0.57   |   14.71   |   2.74   |   1.37   |   40.81   |   3.38   |   1.73   |   43.06   | 1.45 |
+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+------+


Epoch   7 /  10:  80%|███████▉  | 1238/1554 [02:29<00:38,  8.28it/s]
Traceback (most recent call last):
  File "/media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/episodic-memory/NLQ/VSLNet/main.py", line 361, in <module>
    main(configs, parser)
  File "/media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/episodic-memory/NLQ/VSLNet/main.py", line 215, in main
    h_score, start_logits, end_logits = model(
  File "/home/nicolo/ingegneriaMatematica/machineLearning/venv-ia/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1780, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/nicolo/ingegneriaMatematica/machineLearning/venv-ia/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1791, in _call_impl
    return forward_call(*args, **kwargs)
  File "/media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/episodic-memory/NLQ/VSLNet/model/VSLNet.py", line 122, in forward
    query_features = self.e

Error while terminating subprocess (pid=124028): 


TypeError: %d format: a real number is required, not NoneType

In [9]:
%%bash

BASE="/media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning"
REPO="$BASE/episodic-memory/NLQ/VSLNet"
DATA="$BASE/ego4d_data"

PRETRAIN_CKPT="$BASE/checkpoints/omnivore_vslnet_pretrain/vslnet_nlq_official_128_bert/model/best.t7"

MODEL_SUBDIR="$REPO/runs/omnivore_vslnet_finetune_from_pretrain"
mkdir -p "$MODEL_SUBDIR"
cd "$REPO"

python3 main.py \
  --model_name vslnet \
  --task nlq_official_v1_omnivore \
  --predictor bert \
  --dim 128 \
  --mode train \
  --video_feature_dim 1536 \
  --max_pos_len 128 \
  --init_lr 0.0025 \
  --epochs 20 \
  --batch_size 32 \
  --fv official \
  --num_workers 2 \
  --data_loader_workers 1 \
  --model_dir "$MODEL_SUBDIR" \
  --eval_gt_json "$DATA/v1/annotations/nlq_val.json" \
  --log_to_tensorboard "omnivore_vslnet_finetune_bs32_dim128_epoch6_ilr0.0005" \
  --tb_log_freq 5 \
  --remove_empty_queries_from train \
  --pretrained_path "$PRETRAIN_CKPT"


Running with Namespace(save_dir='datasets', task='nlq_official_v1_omnivore', eval_gt_json='/media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/ego4d_data/v1/annotations/nlq_val.json', fv='official', max_pos_len=128, num_workers=2, data_loader_workers=1, word_size=None, char_size=None, word_dim=300, video_feature_dim=1536, char_dim=50, dim=128, highlight_lambda=5.0, num_heads=8, drop_rate=0.2, predictor='bert', gpu_idx='0', seed=12345, mode='train', epochs=20, batch_size=32, num_train_steps=None, init_lr=0.0025, clip_norm=1.0, warmup_proportion=0.0, extend=0.1, period=100, text_agnostic=False, video_agnostic=False, model_dir='/media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/episodic-memory/NLQ/VSLNet/runs/omnivore_vslnet_finetune_from_pretrain', model_name='vslnet', suffix=None, log_to_tensorboard='omnivore_vslnet_finetune_bs32_dim128_epoch6_ilr0.0005', tb_log_dir='./runs', tb_log_freq=5, slurm=False, slurm_wait=False, slurm_partition='pixar', slurm_cons

load video features: 100%|██████████| 1659/1659 [00:08<00:00, 205.55it/s]


Using device=cuda:0
Writing to tensorboard: ./runs/omnivore_vslnet_finetune_bs32_dim128_epoch6_ilr0.0005
Loading pretrained weights from /media/nicolo/2ecaf720-765b-47ee-87b0-10ed6434559d/machineLearning/checkpoints/omnivore_vslnet_pretrain/vslnet_nlq_official_128_bert/model/best.t7
start training...


Epoch   1 /  20:   0%|          | 0/323 [00:00<?, ?it/s]/home/nicolo/ingegneriaMatematica/machineLearning/venv-ia/lib/python3.10/site-packages/transformers/models/bert/modeling_bert.py:413: UserWarning: Mem Efficient attention on Current AMD GPU is still experimental. Enable it with TORCH_ROCM_AOTRITON_ENABLE_EXPERIMENTAL=1. (Triggered internally at /pytorch/aten/src/ATen/native/transformers/hip/sdp_utils.cpp:366.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Epoch   1 /  20:  50%|████▉     | 160/323 [00:16<00:16,  9.80it/s]


Epoch:  1 | Step:   161


...

Epoch  20 /  20: 100%|█████████▉| 322/323 [00:50<00:00,  7.81it/s]


Epoch: 20 | Step:  6460



evaluate val: 100%|██████████| 122/122 [00:08<00:00, 14.90it/s]


Evaluated: 3874 / 3875 instances
+Epoch 20, Step 6460--+-----------+----------+----------+-----------+----------+----------+-----------+------+
|  Rank@1  |  Rank@1  |   Rank@1  |  Rank@3  |  Rank@3  |   Rank@3  |  Rank@5  |  Rank@5  |   Rank@5  | mIoU |
| mIoU@0.3 | mIoU@0.5 | mIoU@0.01 | mIoU@0.3 | mIoU@0.5 | mIoU@0.01 | mIoU@0.3 | mIoU@0.5 | mIoU@0.01 |      |
+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+------+
|   6.30   |   3.59   |   17.68   |  10.38   |   6.35   |   31.47   |  12.60   |   7.90   |   38.85   | 4.72 |
+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+------+


Epoch  20 /  20: 100%|██████████| 323/323 [00:58<00:00,  5.52it/s]
